In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
data=pd.read_csv("../artifacts/sentiment_analysis.csv")

In [3]:
data.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [4]:
data.shape

(7920, 3)

In [5]:
data.duplicated().sum()

np.int64(0)

In [6]:
data.isnull().sum()

id       0
label    0
tweet    0
dtype: int64

In [7]:
##preprossing


In [8]:
import re
import string

In [9]:
 data["tweet"]= data["tweet"].apply(lambda x:" ".join(x.lower() for x in x.split()))

In [10]:
data["tweet"] = data["tweet"].apply(lambda x: " ".join(re.sub(r"^https?:\/\/.*[\r\n]*", '', x, flags=re.MULTILINE) for x in x.split()))


In [11]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

data["tweet"] = data["tweet"].apply(remove_punctuations)


In [12]:
data.head()
data["tweet"].tail(10)


7910    perfect match instagood applewatch red instagr...
7911    i am completely in love with the new iphone em...
7912    tune in turn on drop out  gtd in one app  mobi...
7913    ok so my galaxy crashed after one day now i ha...
7914    gain followers rt this must follow me i follow...
7915    live out loud lol liveoutloud selfie smile son...
7916    we would like to wish you an amazing day make ...
7917    helping my lovely 90 year old neighbor with he...
7918    finally got my smart pocket wifi stay connecte...
7919    apple barcelona apple store bcn barcelona trav...
Name: tweet, dtype: object

In [13]:
data["tweet"] = data['tweet'].str.replace(r'\d+', '', regex=True)


In [14]:
!pip install nltk
import nltk


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
nltk.download('stopwords',download_dir='../static/model')

[nltk_data] Downloading package stopwords to ../static/model...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
with open('../static/model/corpora/stopwords/english', 'r') as file:
    sw = file.read().splitlines()


In [89]:
sww

NameError: name 'sww' is not defined

In [18]:
 data["tweet"]= data["tweet"].apply(lambda x:" ".join(x for x in x.split() if x not in sw))

In [19]:
from nltk.stem import PorterStemmer
ps=PorterStemmer()

In [20]:
 data["tweet"]= data["tweet"].apply(lambda x:" ".join(ps.stem(x)for x in x.split()))

In [21]:
data.head()

,id,label,tweet
0,1,0,fingerprint pregnanc test android app beauti c...
1,2,0,final transpar silicon case thank uncl yay son...
2,3,0,love would go talk makememori unplug relax iph...
3,4,0,im wire know im georg made way iphon cute dave...
4,5,1,amaz servic appl wont even talk question unles...


In [22]:
from collections import Counter
vocab =Counter()

In [23]:
vocab

Counter()

In [24]:
for sentence in data['tweet']:
   vocab.update(sentence.split())

In [90]:
vocab/

SyntaxError: invalid syntax (3450116738.py, line 1)

In [26]:
tokens =[key for key in vocab if vocab[key]>10]

In [91]:
tokens/

SyntaxError: invalid syntax (1556070694.py, line 1)

In [28]:
len(tokens)

1145

In [29]:
def save_vocabulary(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w', encoding="utf-8")
    file.write(data)
    file.close()

save_vocabulary(tokens, '../static/model/vocabulary.txt')

In [30]:
#divide data set

In [31]:
data

,id,label,tweet
0,1,0,fingerprint pregnanc test android app beauti c...
1,2,0,final transpar silicon case thank uncl yay son...
2,3,0,love would go talk makememori unplug relax iph...
3,4,0,im wire know im georg made way iphon cute dave...
4,5,1,amaz servic appl wont even talk question unles...
...,...,...,...
7915,7916,0,live loud lol liveoutloud selfi smile soni mus...
7916,7917,0,would like wish amaz day make everi minut coun...
7917,7918,0,help love year old neighbor ipad morn made rea...
7918,7919,0,final got smart pocket wifi stay connect anyti...


In [32]:
X=data['tweet']
y=data['label']


In [33]:
!pip install scikit-learn



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [34]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2)

In [35]:
X_train.shape

(6336,)

In [36]:
X_test.shape

(1584,)

In [37]:
def vectorizer(ds, vocabulary):
    vectorized_lst = []

    for sentence in ds:
        sentence_lst = np.zeros(len(vocabulary))
        
        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_lst[i] = 1
        
        vectorized_lst.append(sentence_lst)

    vectorized_lst_new = np.asarray(vectorized_lst, dtype=np.float32)
    
    return vectorized_lst_new


In [38]:
 v_X_train=vectorizer(X_train, tokens)

In [39]:
v_X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], shape=(6336, 1145), dtype=float32)

In [40]:
for i in v_X_train[2]:
    print(i)

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [40]:
 v_X_test=vectorizer(X_test, tokens)

In [41]:
v_X_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(1584, 1145), dtype=float32)

In [42]:
y_train.value_counts()

label
0    4738
1    1598
Name: count, dtype: int64

In [45]:
#handle imballence
!pip install imbalanced-learn


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:

!pip install imbalanced-learn

In [46]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
vectorized_X_train_smote, y_train_smote = smote.fit_resample(v_X_train, y_train)

print(vectorized_X_train_smote.shape, y_train_smote.shape)


(9476, 1145) (9476,)


In [49]:
 y_train_smote.value_counts()


label
0    4738
1    4738
Name: count, dtype: int64

In [50]:
vectorized_X_train_smote

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(9476, 1145), dtype=float32)

In [51]:
y_train_smote

0       0
1       0
2       0
3       0
4       0
       ..
9471    1
9472    1
9473    1
9474    1
9475    1
Name: label, Length: 9476, dtype: int64

In [52]:
v_X_test


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(1584, 1145), dtype=float32)

In [53]:
y_test

3601    0
3049    1
3210    0
596     0
6848    0
       ..
677     1
5209    1
5097    1
1296    0
193     0
Name: label, Length: 1584, dtype: int64

In [55]:
y_test.value_counts()

label
0    1156
1     428
Name: count, dtype: int64

In [56]:
#model train and evelution

In [57]:

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC


In [71]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def training_scores(y_act, y_pred):
    acc = round(accuracy_score(y_act, y_pred), 3)
    pr = round(precision_score(y_act, y_pred), 3)
    rec = round(recall_score(y_act, y_pred), 3)
    f1 = round(f1_score(y_act, y_pred), 3)
    print(f'Training Scores:\n\tAccuracy = {acc}\n\tPrecision = {pr}\n\tRecall = {rec}\n\tF1-Score = {f1}')

def validation_scores(y_act, y_pred):
    acc = round(accuracy_score(y_act, y_pred), 3)
    pr = round(precision_score(y_act, y_pred), 3)
    rec = round(recall_score(y_act, y_pred), 3)
    f1 = round(f1_score(y_act, y_pred), 3)
    print(f'Testing Scores:\n\tAccuracy = {acc}\n\tPrecision = {pr}\n\tRecall = {rec}\n\tF1-Score ={f1}')

In [60]:
#LogisticRegression

In [70]:
lr=MultinomialNB()
lr.fit(vectorized_X_train_smote,y_train_smote)
y_train_pred=lr.predict(vectorized_X_train_smote)
y_train_pred

y_test_pred=lr.predict(v_X_test)
y_test_pred

training_scores(y_train_smote,y_train_pred)

validation_scores(y_test, y_test_pred)

Training Scores:
	Accuracy = 0.908
	Precision = 0.87
	Recall = 0.959
	F1-Score = 0.912
Testing Scores:
	Accuracy = 0.872
	Precision = 0.701
	Recall = 0.918
	F1-Score =0.795


In [72]:
lr=LogisticRegression()
lr.fit(vectorized_X_train_smote,y_train_smote)
y_train_pred=lr.predict(vectorized_X_train_smote)
y_train_pred

y_test_pred=lr.predict(v_X_test)
y_test_pred

training_scores(y_train_smote,y_train_pred)

validation_scores(y_test, y_test_pred)

Training Scores:
	Accuracy = 0.937
	Precision = 0.911
	Recall = 0.969
	F1-Score = 0.939
Testing Scores:
	Accuracy = 0.873
	Precision = 0.731
	Recall = 0.839
	F1-Score =0.781


In [74]:
lr=DecisionTreeClassifier()
lr.fit(vectorized_X_train_smote,y_train_smote)
y_train_pred=lr.predict(vectorized_X_train_smote)
y_train_pred

y_test_pred=lr.predict(v_X_test)
y_test_pred

training_scores(y_train_smote,y_train_pred)

validation_scores(y_test, y_test_pred)

Training Scores:
	Accuracy = 1.0
	Precision = 1.0
	Recall = 1.0
	F1-Score = 1.0
Testing Scores:
	Accuracy = 0.814
	Precision = 0.678
	Recall = 0.596
	F1-Score =0.634


In [75]:
lr=RandomForestClassifier()
lr.fit(vectorized_X_train_smote,y_train_smote)
y_train_pred=lr.predict(vectorized_X_train_smote)
y_train_pred

y_test_pred=lr.predict(v_X_test)
y_test_pred

training_scores(y_train_smote,y_train_pred)

validation_scores(y_test, y_test_pred)

Training Scores:
	Accuracy = 1.0
	Precision = 1.0
	Recall = 1.0
	F1-Score = 1.0
Testing Scores:
	Accuracy = 0.867
	Precision = 0.796
	Recall = 0.682
	F1-Score =0.735


In [ ]:
#support vector mechine

In [92]:
sv=SVC()
sv.fit(vectorized_X_train_smote,y_train_smote)
y_train_pred=sv.predict(vectorized_X_train_smote)
y_train_pred

y_test_pred=sv.predict(v_X_test)
y_test_pred

training_scores(y_train_smote,y_train_pred)

validation_scores(y_test, y_test_pred)

Training Scores:
	Accuracy = 0.979
	Precision = 0.962
	Recall = 0.998
	F1-Score = 0.98
Testing Scores:
	Accuracy = 0.881
	Precision = 0.77
	Recall = 0.797
	F1-Score =0.783


In [93]:
import pickle
with open('../static/model/model3 .pickle','wb') as file:
     pickle.dump(sv, file)